In [2]:
import torch
import numpy as np
from torch import nn
from torchvision.ops import roi_align

In [21]:
a = np.ones((3, 16, 16))
b = np.ones((3, 16, 16)) * 2
c = np.ones((3, 16, 16)) * 3
d = np.ones((3, 16, 16)) * 4

A = np.stack([a, b, c, d], axis=0).astype(np.float32)
A = torch.Tensor(A)

In [27]:
B = np.array([[0, 0, 2, 2], [1, 1, 3, 3]], dtype=np.float32)
B = np.stack([B, B, B, B], axis=0)
B = torch.Tensor(B)
B = tuple([b.view(-1, 4) for b in B])

In [23]:
A.shape

torch.Size([4, 3, 16, 16])

In [28]:
B[0].shape

torch.Size([2, 4])

In [32]:
output = roi_align(A, B, (3, 3), 1, 1)

In [33]:
output.shape

torch.Size([8, 3, 3, 3])

In [41]:
output[7]

tensor([[[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.]],

        [[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.]],

        [[4., 4., 4.],
         [4., 4., 4.],
         [4., 4., 4.]]])

In [45]:
C = torch.repeat_interleave(A, 2, dim=0)

In [49]:
C[3]

tensor([[[2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
         [2., 2., 2., 2.,